# Projections in FloodAdapt

In FloodAdapt,  a [Projection](https://deltares-research.github.io/FloodAdapt/api_ref/objects/Projection.html#flood_adapt.objects.Projection) object is used to describe future climate and socio-economic conditions. 

Like all FloodAdapt objects, the Projection has a unique `name` and an optional `description` field. 

The physical part of the Projection is described by the `physical_projection` field which should be a [PhysicalProjection](https://deltares-research.github.io/FloodAdapt/api_ref/objects/PhysicalProjection.html#flood_adapt.objects.PhysicalProjection) object. A PhysicalProjection has the `sea_level_rise` and `subsidence` fields, which are relevant for all types of events, and the `rainfall_multiplier` and `storm_frequency_increase` fields which are applicable only for events sets (risk scenarios).

The socio-economic part of the Projection is described by the `socio_economic_projection` field which should be a [SocioEconomicProjection](https://deltares-research.github.io/FloodAdapt/api_ref/objects/SocioEconomicProjection.html) object. A SocioEconomicProjection has the `population_growth_existing`, `economic_growth`, and `population_growth_new` fields. 
If the `population_growth_new` field is not zero, then the `new_development_elevation` and `new_development_shapefile` should be provided to describe


In this notebook, we will look into all the available FloodAdapt methods to create and manipulate Projections.

# 0. Import libraries

First the required python libraries for this notebook are imported. 

In [ ]:
from flood_adapt import FloodAdapt
from pathlib import Path
from flood_adapt.config.config import Settings
import pandas as pd

# 1. Reading-in the FloodAdapt database

Then, we need to create a `FloodAdapt` object, with the example database of Charleston. This object includes all the required methods for adding, copying, editing or deleting projections from the database. 

In [ ]:
settings = Settings(DATABASE_ROOT=Path(r"c:\Users\athanasi\Github\Database_test"), 
                    DATABASE_NAME="charleston_test", 
                    SYSTEM_FOLDER=Path(r"c:\Users\athanasi\Github\FloodAdapt\flood_adapt\system")
                    )
dbs_path = Path(r"c:\Users\athanasi\Github\Database_test\charleston_test")
fa = FloodAdapt(database_path=dbs_path)

# 2. Getting available projections from the database

Using the `get_projections()` method of the `FloodAdapt` class, we can get a dictionary of all the projections in the database. The keys of the dictionary are the names, descriptions, paths and last_modification_date of the projections. We can use this method to check which projections are available in the database.

In [ ]:
pd.DataFrame(fa.get_projections()) # here we turn the dictionary to a Pandas DataFrame for better visualization

Right now, there is only one projection available in the database named "current". 

We can get the [Projection](https://deltares-research.github.io/FloodAdapt/api_ref/objects/Projection.html#flood_adapt.objects.Projection) object of the projection by using the `get_projection()` method of the `FloodAdapt` class. This method takes the name of the projection as an argument and returns the corresponding Projection object.

In [ ]:
fa.get_projection("current")

# 3. Creating a Projection object


A `Projection` object can be created by using the individual FloodAdapt classes. This can ensure correct type hinting and avoid errors. 


First, let's create simple projection that describes a future conditions with 0.5 meters of sea level rise. In FloodAdapt to define a value with a unit, a [UnitfulLength](https://deltares-research.github.io/FloodAdapt/api_ref/UnitfulLength.html) object can be used which has a `value` and a `unit` field. The `value` field is a float and the `unit` field can be one of the [UnitTypesLength](https://deltares-research.github.io/FloodAdapt/api_ref/UnitTypesLength.html).

In [ ]:
# TODO why is physical_projection and socio_economic_change mandatory? Maybe we can have default values?
# first we need to import the classes we want to use
from flood_adapt.objects.forcing.unit_system import UnitTypesLength, UnitfulLength
from flood_adapt.objects.projections.projections import PhysicalProjection, Projection, SocioEconomicChange

# the we create the projection object
slr_50cm = Projection(name="slr_50cm", 
                    description="0.5 m sea level rise", 
                    physical_projection= PhysicalProjection(sea_level_rise=UnitfulLength(value=0.5, units=UnitTypesLength.meters)), 
                    socio_economic_change=SocioEconomicChange() # we leave this empty for now, since we only do sea level rise
                    )
slr_50cm

We can see that since we did not provide a socio-economic projection, the socio-economic projection fields are set to the default values.

Alternatively,  a new projection can be created by using the `create_projection()` method of the `FloodAdapt` class. This method takes a single argument which is a dictionary containing the required projection parameters. Let's create the exact same projection as before with the 0.5 meters of sea level rise.

In [ ]:
# Create dictionary
slr_50cm_dict = {
    "name": "slr_50cm",
    "description": "0.5 m sea level rise",
    "physical_projection": {"sea_level_rise": {"value": 0.5, "units": "meters"}},
    "socio_economic_change": {}
}
# Create Projection object from dictionary
slr_50cm_from_dict = fa.create_projection(slr_50cm_dict)

# Check if the two objects are equal
slr_50cm_from_dict == slr_50cm

As you can see the two projections are identical. 

# 4. Saving a new Projection to the database

In the previous step we have created a `Projection` object, but we have not yet saved it in our database. The `save_projection()` method of the `FloodAdapt` class can be used to save a new projection to the database. This method takes a single argument which is a `Projection` object. If a projection with the same name already exists in the database, an error will be raised. Let's save the projection we just created to the database.

In [ ]:
fa.save_projection(slr_50cm)

Using the `get_projections()` method of the `FloodAdapt` class, we can check that the projection has been saved to the database.

In [ ]:
pd.DataFrame(fa.get_projections())

# 5. Copying and Editing a Projection in the database

If we want to edit small parts of a projection, it is easier to copy an existing projection and edit the copy. This way we do not have to create a new projection from scratch.

A projection can be copied in the database by using the `copy_projection()` method of the `FloodAdapt` class. This method takes three arguments: the name of the projection to be copied and the name and description of the new projection. Let's copy the projection we just created, having in mind that we want to make a projection for 1 meter of sea level rise. 

In [ ]:
# TODO why is it called old_name? we are not replacing the name. This should change to name
fa.copy_projection(old_name="slr_50cm", new_name="slr_1m", new_description="1 m sea level rise")

We can see that now a new projection with name "slr_1m" has been created in the database. However, the actual attributes of the projection are still the same as the original projection. 

In [ ]:
slr_1m = fa.get_projection("slr_1m")
slr_1m.physical_projection.sea_level_rise

We can directly edit the relevant attributes of the projection object. In this case, we want to change the sea level rise to 1 meter.

In [ ]:
slr_1m.physical_projection.sea_level_rise.value = 1.0

Then using the `edit_projection()` method of the `FloodAdapt` class, we can save the changes to the database. This method takes a single argument which is a `Projection` object. The `name` field of the projection object provided will be used to identify which projection is going to be updated in the database, with the given Projection object attributes.

In [ ]:
fa.edit_projection(slr_1m)

Now we can verify that the projection has been updated in the database. The sea level rise is now 1 meter.

In [ ]:
slr_1m = fa.get_projection("slr_1m")
slr_1m.physical_projection.sea_level_rise

# 6. Deleting a Projection from the database

We now have 3 projections in the database.

In [ ]:
pd.DataFrame(fa.get_projections())


If we want to delete a projection from the database, we can use the `delete_projection()` method of the `FloodAdapt` class. This method takes a single argument which is the name of the projection to be deleted. Let's delete the projection we just created.

In [ ]:
fa.delete_projection("slr_1m")

We can check that the projection has been indeed deleted from the database by using the `get_projections()` method of the `FloodAdapt` class. The projection with name "slr_1m" is no longer in the database.

In [ ]:
pd.DataFrame(fa.get_projections())

# 7. Use of Sea Level Rise Scenarios

A FloodAdapt database can include sea level rise scenarios, describing a timeline of future sea level rise relative to a reference year. If these scenarios are available in the database the `get_slr_scn_names()` method  will return a list of the available scenarios.

In [ ]:
fa.get_slr_scn_names()

In [ ]:
# TODO the display is not working in the notebook. We need to check why this is the case.
from IPython.display import IFrame, display
from pathlib import Path

# Ensure the path is correct and accessible
html_path = fa.plot_slr_scenarios()

display(IFrame(src=html_path, width='500px', height='200px'))

Then the `interp_slr()` method can be used to interpolate the sea level rise for a given year. This method takes two arguments: the name of the scenario and the year for which we want to interpolate the sea level rise. The method returns a float value representing the interpolated sea level rise in length units of the database.

In [ ]:
# TODO why this is unitless?
fa.interp_slr(slr_scenario="ssp585", year=2050)

In [ ]:
# TODO why call it population_growth_existing? No need to
# TODO add documentation for units
# TODO if population_growth_new is given, shapefile should be mandatory 
proj_2050 = Projection(name="proj_2050",
                    description="2050 projection", 
                    physical_projection= PhysicalProjection(sea_level_rise=UnitfulLength(value=fa.interp_slr(slr_scenario="ssp585", year=2050), units=UnitTypesLength.meters)), 
                    socio_economic_change=SocioEconomicChange(population_growth_existing=10,
                                                              economic_growth=5)
                    )
proj_2050

In [ ]:
fa.get_building_geometries().explore()

In [ ]:
import geopandas as gpd
new_dev = gpd.read_file("data/new_dev.geojson")
new_dev.explore()

In [ ]:
proj_2050.socio_economic_change = SocioEconomicChange(population_growth_existing=10,
                                                              economic_growth=5,
                                                              population_growth_new=5,
                                                              new_development_shapefile="data/new_dev.geojson")

In [ ]:
fa.save_projection(proj_2050)

In [ ]:
fa.get_projection("proj_2050")